# **Caso 3.1: Preparando los Datos**


### Creamos la conexión con Vantage

In [1]:
%connect DataAnalyst

Password: ·······


Success: 'DataAnalyst' connection established and activated for user 'pocuser', with default database 'DW_LANDING'


### Explorando las Tablas Originales

In [2]:
SELECT COUNT(1) N, COUNT(DISTINCT CAT_001) NID FROM DW_LANDING.caso3_df1;

,N,NID
1,118154,117821


In [3]:
SELECT COUNT(1) N, COUNT(DISTINCT CAT_011) NID FROM DW_LANDING.caso3_df2;

,N,NID
1,322399,319842


In [4]:
SELECT COUNT(1) N, COUNT(DISTINCT CAT_016) NIDREC, COUNT(DISTINCT CAT_011) NID FROM DW_LANDING.caso3_df3;

,N,NIDREC,NID
1,74000000,996730,319843


In [5]:
SELECT COUNT(1) N, COUNT(DISTINCT CAT_016) NIDREC, COUNT(DISTINCT CAT_011) NID, COUNT(DISTINCT CAT_031) NID3 FROM DW_LANDING.caso3_df4;

,N,NIDREC,NID,NID3
1,60000000,996708,319885,19004400


### Creando DF5

In [6]:
REPLACE VIEW DW_LANDING.caso3_df5 AS
SELECT CAT_001 E_CAT016, CAT_002 E_CAT_002, CAT_003 E_CAT_003, CAT_004 E_CAT_004, CAT_006 E_CAT_006, CAST(CAT_007 AS VARCHAR(10)) E_CAT_007, 
CAST(CAT_008 AS VARCHAR(10)) E_CAT_008, substring(CAT_001,1,2) G_CAT001 FROM DW_LANDING.caso3_df1;

Success: 9 rows affected

### Creando DF6

In [7]:
REPLACE VIEW DW_LANDING.caso3_df6 AS
SELECT CAT_011, CAT_002 R_CAT_002, CAT_003 R_CAT_003, CAT_004 R_CAT_004, CAT_006 R_CAT_006, 
CAT_007 R_CAT_007, CAT_008 R_CAT_008, CAT_009 R_CAT_009, CAT_010 R_CAT_010
FROM DW_LANDING.caso3_df2
WHERE R_CAT_006 IN ('PPP MMM CCC','PPP MMM');

Success: 10 rows affected

### Creando DF7

In [8]:
CREATE MULTISET TABLE DW_LANDING.caso3_df7 AS (
SELECT CAT_015, CAT_016, CAT_017, CAT_018, CAT_019, CAT_020, DF3.CAT_011, CAT_021, CAT_022, CAST(CAT_023 AS DATE) CAT_023, 
TO_CHAR(CAT_023, 'DAY') VATP003, E_CAT_003, G_CAT001, E_CAT_006, E_CAT_007, CAST(CAT_024 AS FLOAT) CAT_024, CAT_025, CAT_026, CAT_027, R_CAT_010, E_CAT016, 
CASE WHEN CAT_024>=3500 THEN 'TRUE' ELSE 'FALSE' END AS CAT_024_3500
FROM DW_LANDING.caso3_df3 DF3
LEFT JOIN DW_LANDING.caso3_df5 DF5 ON DF3.CAT_016=DF5.E_CAT016
LEFT JOIN DW_LANDING.caso3_df6 DF6 ON DF3.CAT_011=DF6.CAT_011
WHERE YEAR(CAT_023)=2021 AND LENGTH(DF3.CAT_011)=11) WITH DATA PRIMARY INDEX(CAT_016);

Success: 0 rows affected

### Creando DF8

In [9]:
REPLACE VIEW DW_LANDING.caso3_df8 AS
SELECT CAT_016, CAT_023, COUNT(CAT_016) CANT_001, SUM(CAT_024) SUM_001 FROM (
SELECT CAT_016, CAT_024, CAST(DATE '1970-01-01' + (CAT_023 / 86400000) AS DATE) CAT_023, '0'||CAST(CAT_017 AS VARCHAR(2)) CAT_017 FROM DW_LANDING.caso3_df4) AS DT
WHERE (CAT_023 BETWEEN DATE '2021-01-01' AND DATE '2023-03-31') AND (CAT_024 BETWEEN 0 AND 999999999) AND (CAT_017='03')
GROUP BY 1,2;

Success: 5 rows affected

### Creando df7_time

In [10]:
REPLACE VIEW DW_LANDING.caso3_df7_time AS
SELECT CAT_016, CAT_023, VATP003, E_CAT_003, G_CAT001, E_CAT_006, E_CAT_007, sum(CAT_024) as VATP016_N, count(CAT_024) VATP012_N,
max(CAT_024) VATP010_N, count(Distinct CAT_011) VATP014_N, count(Distinct CAT_015) VATA_01, count(Distinct R_CAT_010) VATA_02, count(Distinct CAT_018) VATA_03
FROM DW_LANDING.caso3_df7
GROUP BY 1,2,3,4,5,6,7;

Success: 15 rows affected

### Creando df7_grouped_mode

In [11]:
REPLACE VIEW DW_LANDING.caso3_df7_grouped_mode AS
SEL CAT_016, CAT_023, MAX(C_CAT_024) VATP008, MAX(S_CAT_024) VATP009_N FROM
(SEL CAT_016, CAT_023, E_CAT016, count(CAT_024) C_CAT_024, sum(CAT_024) S_CAT_024 
FROM DW_LANDING.caso3_df7 GROUP BY 1, 2, 3) AS DT
GROUP BY 1, 2;

Success: 5 rows affected

### Creamos la Tabla Final, cruzando las anteriores

In [12]:
DROP TABLE DW_LANDING.caso3_final;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'DW_LANDING.caso3_final' does not exist.

In [13]:
CREATE MULTISET TABLE DW_LANDING.caso3_final AS (
SELECT DF7.CAT_016 LF001, DF7.CAT_023 VATP001, TD_DAY_OF_MONTH(DF7.CAT_023) VATP002, 
VATP003, E_CAT_003 VATP004_Index, G_CAT001 VATP005_Index, E_CAT_006 VATP006_Index, E_CAT_007 VATP007_Index, VATP008, VATP009_N, VATP010_N, 
VATP016_N/(VATP016_N+SUM_001) VATP011_N, VATP012_N, VATP012_N/(VATP012_N+CANT_001) VATP013_N, VATP014_N, 
CASE WHEN YEAR(DF7.CAT_023)=2021 THEN (VATP016_N-2200)/4400 
WHEN YEAR(DF7.CAT_023)=2022 THEN (VATP016_N-2300)/4600 
WHEN YEAR(DF7.CAT_023)=2023 THEN (VATP016_N-2450)/4900 
ELSE 0 END AS VATP015_N, VATP016_N, 
VATP016_N-prev_2 VATP017_N, VATP012_N-prev_3 VATP018_N, VATP014_N-prev_4 VATP019_N, VATA_03-prev_5 VATP020_N,
LAG(VATP016_N) OVER (PARTITION BY DF7.CAT_016 ORDER BY DF7.CAT_023) prev_2, 
LAG(VATP012_N) OVER (PARTITION BY DF7.CAT_016 ORDER BY DF7.CAT_023) prev_3, 
LAG(VATP014_N) OVER (PARTITION BY DF7.CAT_016 ORDER BY DF7.CAT_023) prev_4, 
LAG(VATA_03) OVER (PARTITION BY DF7.CAT_016 ORDER BY DF7.CAT_023) prev_5
FROM DW_LANDING.caso3_df7_time DF7
LEFT JOIN DW_LANDING.caso3_df7_grouped_mode DF7g ON DF7.CAT_016=DF7g.CAT_016 AND DF7.CAT_023=DF7g.CAT_023
LEFT JOIN DW_LANDING.caso3_df8 DF8 ON DF7.CAT_016=DF8.CAT_016 AND DF7.CAT_023=DF8.CAT_023
) WITH DATA PRIMARY INDEX(LF001);

Success: 0 rows affected

### Exploramos los Datos

In [14]:
-- Primeros registros
SEL TOP 10 * FROM caso3_final;

,LF001,VATP001,VATP002,VATP003,VATP004_Index,VATP005_Index,VATP006_Index,VATP007_Index,VATP008,VATP009_N,VATP010_N,VATP011_N,VATP012_N,VATP013_N,VATP014_N,VATP015_N,VATP016_N,VATP017_N,VATP018_N,VATP019_N,VATP020_N,prev_2,prev_3,prev_4,prev_5
1,ID_14861175,2021-04-19,19,MONDAY,,,,,17,860046,95198,,17,,17,195,860046,,,,,,,,
2,ID_12211593,2021-04-19,19,MONDAY,,,,,25,1239559,96872,,25,,25,281,1239559,,,,,,,,
3,ID_12211593,2021-07-28,28,WEDNESDAY,,,,,10,508133,94227,,10,,10,115,508133,-731426,-15,-15,0,1239559,25,25,5
4,ID_29549150,2021-04-19,19,MONDAY,,,,,12,478761,94962,,12,,12,108,478761,,,,,,,,
5,ID_18500819,2021-04-19,19,MONDAY,,,,,18,818517,92576,,18,,17,186,818517,,,,,,,,
6,ID_18500819,2021-07-28,28,WEDNESDAY,,,,,11,563868,83461,,11,,11,128,563868,-254649,-7,-6,0,818517,18,17,5
7,ID_29549150,2021-07-28,28,WEDNESDAY,,,,,11,573131,98390,,11,,11,130,573131,94370,-1,-1,0,478761,12,12,4
8,ID_14861175,2021-07-28,28,WEDNESDAY,,,,,14,666146,87506,,14,,14,151,666146,-193900,-3,-3,-1,860046,17,17,5
9,ID_24259826,2021-04-19,19,MONDAY,,,,,11,551263,92643,,11,,11,125,551263,,,,,,,,
10,ID_24259826,2021-07-28,28,WEDNESDAY,,,,,18,737130,84056,,18,,17,167,737130,185867,7,6,1,551263,11,11,4


In [15]:
-- Cantidad de Registros
SEL COUNT(1) N FROM DW_LANDING.caso3_final;

,N
1,1993493


In [16]:
UPDATE DW_LANDING.caso3_final SET VATP011_N=0 WHERE VATP011_N IS NULL;

Success: 1894287 rows affected

In [17]:
UPDATE DW_LANDING.caso3_final SET VATP013_N=0 WHERE VATP013_N IS NULL;

Success: 1894287 rows affected

In [18]:
UPDATE DW_LANDING.caso3_final SET VATP017_N=0 WHERE VATP017_N IS NULL;

Success: 996730 rows affected

In [19]:
UPDATE DW_LANDING.caso3_final SET VATP018_N=0 WHERE VATP018_N IS NULL;

Success: 996730 rows affected

In [20]:
UPDATE DW_LANDING.caso3_final SET VATP019_N=0 WHERE VATP019_N IS NULL;

Success: 996730 rows affected

In [21]:
UPDATE DW_LANDING.caso3_final SET VATP020_N=0 WHERE VATP020_N IS NULL;

Success: 996730 rows affected

In [22]:
UPDATE DW_LANDING.caso3_final SET VATP017_N=VATP017_N*-1 WHERE VATP017_N<0;

Success: 498748 rows affected

In [23]:
UPDATE DW_LANDING.caso3_final SET VATP018_N=VATP018_N*-1 WHERE VATP018_N<0;

Success: 462315 rows affected

In [24]:
UPDATE DW_LANDING.caso3_final SET VATP019_N=VATP019_N*-1 WHERE VATP019_N<0;

Success: 461560 rows affected

In [25]:
UPDATE DW_LANDING.caso3_final SET VATP020_N=VATP020_N*-1 WHERE VATP020_N<0;

Success: 181732 rows affected

### Estadísticas de las variables numéricas

In [26]:
call val.td_analyze ('statistics',
'database = DW_LANDING;
tablename = caso3_final;
statsoptions = all;
extendedoptions = all;
columns = allnumeric;');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,xdb,xtbl,xcol,xcnt,xmin,xmax,xmean,xstd,xskew,xkurt,xste,xcv,xvar,xsum,xuss,xcss,xtype,xnull,xnique,xblank,xzero,xpos,xneg,xmode,xmode_cnt,xmode_pct,xnbrmodes,xpctile0,xpctile1,xpctile2,xpctile3,xpctile4,xpctile5,xpctile6,xpctile7,xpctile8,xpctile9,xpctile10,xpctile20,xpctile25,xpctile30,xpctile33,xpctile40,xpctile50,xpctile60,xpctile67,xpctile70,xpctile75,xpctile80,xpctile90,xpctile91,xpctile92,xpctile93,xpctile94,xpctile95,xpctile96,xpctile97,xpctile98,xpctile99,xpctile100,xmin_1,xmin_2,xmin_3,xmin_4,xmin_5,xmincnt_1,xmincnt_2,xmincnt_3,xmincnt_4,xmincnt_5,xmax_5,xmax_4,xmax_3,xmax_2,xmax_1,xmaxcnt_5,xmaxcnt_4,xmaxcnt_3,xmaxcnt_2,xmaxcnt_1
1,DW_LANDING,caso3_final,prev_2,996763,820,2.202341e+06,754165.2996359215,228041.9928381943,0.3471382411650106,0.14068546285379468,228.41197727601224,30.2376671199647,5.200315049761506e+10,7.51724066561e+11,6.187590222009642e+17,5.183481629945428e+16,"DECIMAL(15,0)",996730,588189,,0,996763,0,,996730,49.999172307101155,1,820,285175,330808,360777,383655,402628,419116,433751,446869,459261,470793,558267,593110,625010,643152,684216,741103,799691,843730,864052,900575,941730,1.05434e+06,1.06976e+06,1.08654e+06,1.105328e+06,1.126401e+06,1.150252e+06,1.178935e+06,1.214591e+06,1.262474e+06,1.340611e+06,2.202341e+06,820,15597,19206,25098,28200,1,1,1,1,1,2.112461e+06,2.121428e+06,2.13447e+06,2.143599e+06,2.202341e+06,1,1,1,1,1
2,DW_LANDING,caso3_final,prev_3,996763,1,40,15.079725070051758,3.944802142903179,0.2640317480654231,0.07093860920011556,0.003951202347466295,26.1596423315272,15.561463946653515,1.5030912e+07,2.42173112e+08,1.5511091487858197e+07,INTEGER,996730,39,,0,996763,0,,996730,49.999172307101155,1,1,7,8,8,9,9,9,9,10,10,10,12,12,13,13,14,15,16,17,17,18,18,20,20,21,21,21,22,22,23,24,25,40,1,2,3,4,5,6,45,201,652,2052,35,36,37,38,40,8,1,3,2,1
3,DW_LANDING,caso3_final,prev_4,996763,1,37,14.843217495031416,3.8527945605700755,0.2542265947277567,0.05970195299293574,0.0038590454883565732,25.956599786129598,14.844025925958363,1.479517e+07,2.34403902e+08,1.4795975814036034e+07,INTEGER,996730,37,,0,996763,0,,996730,49.999172307101155,1,1,7,8,8,8,9,9,9,10,10,10,12,12,13,13,14,15,16,16,17,17,18,20,20,20,21,21,21,22,22,23,24,37,1,2,3,4,5,6,45,204,694,2172,33,34,35,36,37,12,13,4,2,1
4,DW_LANDING,caso3_final,prev_5,996763,1,5,4.7420480094064485,0.49441524160581113,-1.8129413910483512,2.8973614109866164,0.0004952174006421249,10.426196458261838,0.24444643113213257,4.726698e+06,2.2657884e+07,243655.15803455785,INTEGER,996730,5,,0,996763,0,,996730,49.999172307101155,1,1,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,2,3,4,5,41,1229,22498,208270,764725,1,2,3,4,5,41,1229,22498,208270,764725
5,DW_LANDING,caso3_final,VATP002,1.993493e+06,19,28,23.49999774265573,4.499999999999434,1.0032656296243013e-06,-2.0000010032628617,0.003187169467272224,19.148938009603782,20.249999999994905,4.6847081e+07,1.141274531e+09,4.0368233249989845e+07,INTEGER,0,2,,0,1.993493e+06,0,19,996747,50.00002508160299,1,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,19,28,,,,996747,996746,,,,,,,19,28,,,,996747,996746
6,DW_LANDING,caso3_final,VATP008,1.993493e+06,1,50,15.084497913963078,3.9460077936457476,0.27152511743875724,0.10166818257930779,0.0027947990127838987,26.15935787954265,15.570977507512982,3.0070841e+07,4.84644173e+08,3.1040634664384574e+07,INTEGER,0,44,,0,1.993493e+06,0,15,201528,10.109290576891917,1,1,7,8,8,9,9,9,9,10,10,10,12,12,13,13,14,15,16,17,17,18,18,20,20,21,21,21,22,22,23,24,25,50,1,2,3,4,5,9,81,408,1357,4109,40,42,46,48,50,9,2,2,2,2
7,DW_LANDING,caso3_final,VATP009_N,1.993493e+06,820,2.799044e+06,754277.2909195066,227966.39202672153,0.35261004460777423,0.16117289211160826,161.4594498293041,30.2231546370457,5.196867589368089e+10,1.503646499507e+12,1.2377655997620608e+18,1.035991916133216e+17,"DECIMAL(15,0)",0,806045,,0,1.993493e+06,0,823583,15,0.0007524480898603607,1,820,285881,331249,361163,384278,403180,419778,434367,447562,459914,471323,558695,59333

### Anomaly Detection con OneClassSVM

In [ ]:
CREATE VIEW caso3_train AS SEL * FROM caso3_final SAMPLE 90000;

In [ ]:
CREATE MULTISET TABLE oneclasssvm_model AS(
SELECT * FROM TD_OneClassSVM(
  ON caso3_train AS InputTable
  USING
    InputColumns('VATP009_N','VATP010_N','VATP011_N','VATP015_N','VATP016_N','VATP017_N')
    Tolerance(1e-7)
    BatchSize(30)
    LearningRate('constant')
    InitialEta (0.01)
    RegularizationLambda(0.1)
    Alpha(0)
    Momentum (0.0)
    Nesterov ('false')
    MaxIterNum (150)
) AS dt
) WITH DATA;

In [ ]:
CREATE MULTISET TABLE caso3_onesvm AS (
    SELECT * from TD_OneClassSVMPredict(
        ON caso3_final AS InputTable PARTITION BY ANY
        ON oneclasssvm_model AS ModelTable DIMENSION
        USING
        IDColumn('index_label')
        Accumulate('tar')
        OutputProb('t') 
    ) AS dt
) WITH DATA;

In [ ]:
SEL TOP 10 * FROM caso3_onesvm;

In [ ]:
SEL prediction, COUNT(1) N FROM caso3_onesvm GROUP BY 1;

### Limpiando las tablas creadas

In [27]:
DROP TABLE DW_LANDING.caso3_df7;

Success: 37 rows affected

In [ ]:
DROP TABLE DW_LANDING.caso3_train;

In [ ]:
DROP TABLE DW_LANDING.caso3_onesvm;

Copyright 2023. Elaborado por Luis Cajachahua bajo licencia MIT